# ①ライブラリのインポート

In [1]:
# データハンドリング
import numpy as np
import pandas as pd
# 正規化
from sklearn.preprocessing import MinMaxScaler
# データ分割
from sklearn.model_selection import train_test_split
# ニューラルネット
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
# 混同行列可視化用
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
# 正解率の算出
from sklearn.metrics import accuracy_score
# 決定木可視化用
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
# グリッドサーチ用
from sklearn import model_selection

# ②データの読み込みと分割

In [2]:
# 読み込み
data=pd.read_csv('data_complete.csv')

#説明変数と目的変数に分割
X=data[
    [
        'Pclass', 'Age', 'SibSp', 'Parch', 
        'Fare', 'Sex_female','Sex_male', 
        'Embarked_C', 'Embarked_Q', 'Embarked_S'
    ]
]
y=data['Survived']

# 訓練データとテストデータに分割
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234)

# 正規化
scaler_x = MinMaxScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test) 

# ③パラメータ調整前

In [3]:
# モデル生成
clf=MLPClassifier(
    random_state=1234
)
# 最適化
clf.fit(X_train,y_train)
# 予測（訓練）
y_train_predict=clf.predict(X_train)
# 正答率(訓練)
print("正解率:",round(accuracy_score(y_train, y_train_predict),3))
# 予測（テスト）
y_test_predict=clf.predict(X_test)
# 正答率(テスト)
print("正解率:",round(accuracy_score(y_test, y_test_predict),3))

正解率: 0.826
正解率: 0.844


c:\users\aika\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# ④ハイパーパラメータ調整Ⅰ

### 調整の際のポイント

「学習率」は0.005~0.01で固定

「学習回数」は、学習曲線の変化がなくなる回数を見て固定（今回は300回くらいでひと先ず固定）

次に「層の数」→「ユニット数」→細かいパラメータの順で決めていく

＊これは、ニューラルネットの知識が浅い方がパラメータチューニングを行う際のTIPSとしてご理解ください。

＊本当なら、活性化関数やバイアス荒野重みの初期値などを問題ごとに適切なものを選定します。

In [4]:
"""基本モデルの作成：分類
"""
clf=MLPClassifier(
    random_state=1234
)

In [5]:
"""探索開始
最適な層の数を探索
"""
clf_cv=model_selection.GridSearchCV(
    clf,
    {
        "learning_rate_init":[0.005], #学習率
        "max_iter":[300], #学習回数
        "hidden_layer_sizes":[(64,64),(64,64,64),(64,64,64,64),(64,64,64,64,64)] # 中間層
    },
)
clf_cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=1234, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_gr

In [6]:
"""最適なパラメータの値
「層の数は4層くらいがいいらしい」ことがわかった
"""
clf_cv.best_params_

{'hidden_layer_sizes': (64, 64, 64, 64, 64),
 'learning_rate_init': 0.005,
 'max_iter': 300}

In [7]:
"""最適なパラメータで再学習＆正解率算出
"""
# モデル生成
clf=MLPClassifier(**clf_cv.best_params_,random_state=1234)
# 最適化
clf.fit(X_train,y_train)
# 予測
y_train_predict=clf.predict(X_train)
# 精度の評価
print("正解率:",round(accuracy_score(y_train, y_train_predict),3))

正解率: 0.847


In [8]:
"""テストデータへの適用
"""
y_test_predict=clf.predict(X_test)
print("正解率:",round(accuracy_score(y_test, y_test_predict),3))

正解率: 0.832


# ⑤ハイパーパラメータ調整Ⅱ

In [9]:
"""基本モデルの作成：分類
"""
clf=MLPClassifier(
    random_state=1234
)

In [10]:
"""探索開始
「層の数は4層くらいがいいらしい」ことがわかっているので、
つぎは、ユニット数を調べてみる。
"""
clf_cv=model_selection.GridSearchCV(
    clf,
    {
        "learning_rate_init":[0.005], #学習率
        "max_iter":[300], #学習回数
        "hidden_layer_sizes":[(16,16,16,16),(32,32,32,32),(64,64,64,64),(128,128,128,128),(256,256,256,256)] # 中間層
    },
)
clf_cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_st...
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'hidden_layer_si

In [11]:
"""最適なパラメータの値
今回のような説明変数の少ないデータでは、「ユニット数は16近辺がいいらしい」ことがわかった
"""
clf_cv.best_params_

{'hidden_layer_sizes': (16, 16, 16, 16),
 'learning_rate_init': 0.005,
 'max_iter': 300}

In [12]:
"""探索開始
「各層のユニット数は16近辺がいいらしい」ということはわかったが、
入力層→出力層にいくユニット数の増減具合はどれがいいんでしょうか

・単調
・山なり
・増加
・減少
"""
clf_cv=model_selection.GridSearchCV(
    clf,
    {
        "learning_rate_init":[0.005], #学習率
        "max_iter":[300], #学習回数
        "hidden_layer_sizes":[(16,16,16,16),(8,16,16,8),(8,12,16,24),(24,16,12,8)] # 中間層
    },
)
clf_cv.fit(X_train,y_train)

c:\users\aika\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=1234, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_gr

In [13]:
"""最適なパラメータの値
「ユニット数の増減具合は、単調がいい」ことがわかった
"""
clf_cv.best_params_

{'hidden_layer_sizes': (16, 16, 16, 16),
 'learning_rate_init': 0.005,
 'max_iter': 300}

### 以上より下記のことがわかったので、再学習してみる

・「層の数は4層くらいがいいらしい」

・「各層のユニット数は16近辺がいいらしい」

・「ユニット数の増減具合は、単調がいい」

In [14]:
"""最適なパラメータで再学習＆正解率算出
"""
# モデル生成
clf=MLPClassifier(**clf_cv.best_params_,random_state=1234)
# 最適化
clf.fit(X_train,y_train)
# 予測
y_train_predict=clf.predict(X_train)
# 精度の評価
print("正解率:",round(accuracy_score(y_train, y_train_predict),3))

正解率: 0.846


In [15]:
"""テストデータへの適用
"""
y_test_predict=clf.predict(X_test)
print("正解率:",round(accuracy_score(y_test, y_test_predict),3))

正解率: 0.838


# 結論

③パラメータ調整前
正解率（訓練データ）: 0.826
正解率（テストデータ）: 0.844

④ハイパーパラメータ調整Ⅰ
正解率（訓練データ）: 0.847
正解率（テストデータ）: 0.832

⑤ハイパーパラメータ調整Ⅱ
正解率（訓練データ）: 0.846
正解率（テストデータ）: 0.838

テストデータの正答率が、「パラメータ調整前」が最良値なのは、ご愛敬として。

パラメータチューニングを丁寧に行うと、段階が進むごとに「モデルがよくなってきている感」がわかると思います。

＊本来は、訓練データの方が精度がよくなるはずです（③のようなことは起こらない→学習不足と考えられる）
＊このような少ないデータを扱う際は、こういうイレギュラーなことも起こります。

### ⑤のモデルで完成形になります！！

### お疲れさまでした。